# Explore Results

In [1]:
import sys

%load_ext autoreload
%autoreload 2

In [52]:
from pathlib import Path
import pandas as pd
import os
import joblib
import pickle
from src.sklearn_utils import predict_fn, plot_one_vs_rest_success_rates

In [3]:
artifacts_dir = Path(os.path.abspath('')).parent / 'artifacts'
model_path = artifacts_dir / 'gradient_boosting-20220704-153024.joblib'
model = joblib.load(model_path)

In [4]:
model.cv_results_

{'mean_fit_time': array([ 21.69707355,  64.43889155, 132.12126284]),
 'std_fit_time': array([ 0.50096234,  1.39458765, 49.39470151]),
 'mean_score_time': array([0.06359849, 0.0473938 , 0.04681373]),
 'std_score_time': array([0.01985951, 0.00117783, 0.00224644]),
 'param_clf__base_estimator__max_depth': masked_array(data=[2, 4, 6],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__base_estimator__max_depth': 2},
  {'clf__base_estimator__max_depth': 4},
  {'clf__base_estimator__max_depth': 6}],
 'split0_test_score': array([0.07306397, 0.07205387, 0.07474747]),
 'split1_test_score': array([0.05938567, 0.05699659, 0.05836177]),
 'split2_test_score': array([0.06622517, 0.06390728, 0.06523179]),
 'split3_test_score': array([0.05785953, 0.05785953, 0.05719064]),
 'split4_test_score': array([0.0620339 , 0.06372881, 0.06237288]),
 'mean_test_score': array([0.06371365, 0.06290922, 0.06358091]),
 'std_test_score': array([0.00546919, 0

In [5]:
mlb = pickle.load(open(artifacts_dir / 'gradient_boosting-20220704-153024-binarizer.pkl', 'rb'))

In [6]:
df = pd.read_csv(artifacts_dir / 'gradient_boosting-20220704-153024-test.csv')
# convert strings in column to list. e.g. "['A', 'B', 'C']" -> ['A', 'B', 'C'], "['Hello', 'World']" -> ['Hello', 'World']
df['sectors_list'] = df['sectors_list'].apply(lambda x: eval(x))
df["sectors_list"] = df["sectors_list"].apply(
    lambda x: [tag for tag in x if tag in mlb.classes_.tolist()]
)

In [7]:
y_true, y_pred, hl = predict_fn(df, mlb, model)

In [25]:
naive_path = artifacts_dir / 'naive-stratified.joblib'
naive_model = joblib.load(naive_path)

In [26]:
y_pred_naive = naive_model.predict(df)

array([0.74045802, 0.71755725, 0.50127226, 0.83969466, 0.77862595,
       0.66666667, 0.72010178, 0.5216285 , 0.81170483, 0.74554707])

In [53]:
plot_one_vs_rest_success_rates(y_true, y_pred, y_pred_naive, classes=mlb.classes_)